# HW06: ML and Causal Inference (due November 9th)

**As a first part of THIS WEEK'S homework (HW06) you will have to provide feedback to two of your classmates' essays on Eduflow.** On Eduflow, you will be automatically assigned to the two essays you have to provide feedback to on Friday, in case you want to start ahead.

### For the coding part of the homework, you will have to solve only one of the three exercises.

## Exercise 1: Double Machine learning with XGboost

In this exercise you will investigate a research question similar to the one in HW02. Namely, what is the effect of being in a Union on wages? You will use the same data as in HW02 (description of the variables can be found [here](https://rdrr.io/rforge/sampleSelection/man/nlswork.html)). Although, here instead of just including controls, you will estimate the effect of union membership on wages using double machine learning and you will use XGboost as machine learning algorithm. The regression of reference is the following: 

<center> $ln\_wage_i = \beta_0 + \beta_1 union_i +\varepsilon_i$ </center>

In [6]:
# Load Data (NLSY)
import pandas as pd
df = pd.read_stata('http://www.stata-press.com/data/r16/nlswork.dta')
df = df.dropna()

In [9]:
df.head()
df.columns

Index(['idcode', 'year', 'birth_yr', 'age', 'race', 'msp', 'nev_mar', 'grade',
       'collgrad', 'not_smsa', 'c_city', 'south', 'ind_code', 'occ_code',
       'union', 'wks_ue', 'ttl_exp', 'tenure', 'hours', 'wks_work', 'ln_wage'],
      dtype='object')

In [21]:
import numpy as np
Y = df['ln_wage']
D = df['union']

In [15]:
import statsmodels.formula.api as smf

## Run the regression written above
res = smf.ols("ln_wage ~ union", data = df ).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ln_wage   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     741.9
Date:                Mon, 07 Nov 2022   Prob (F-statistic):          4.56e-159
Time:                        14:43:06   Log-Likelihood:                -8266.7
No. Observations:               13452   AIC:                         1.654e+04
Df Residuals:                   13450   BIC:                         1.655e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6564      0.004    377.147      0.000       1.648       1.665
union          0.2502      0.009     27.238      0.000       0.232       0.268
==============================================================================
Omnibus:                      425.572   Durbin-Watson:                   1.032
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              895.660
Skew:                           0.205   Prob(JB):                    3.24e-195
Kurtosis:                       4.196   Cond. No.                         2.53
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**What could be potential sources of bias (i.e. confounders)?**

In [45]:
# fill in X with all predictors that are not colliders
X = df[['age', 'year', 'collgrad', 'race', 'msp', 'not_smsa', "c_city", "south"]]

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
cat_vars = ["race", "not_smsa"]
X[cat_vars] = encoder.fit_transform(X[cat_vars])
#df2[cont_vars] = df2[cont_vars].astype('float32')

C:\Users\joppe\AppData\Local\Temp/ipykernel_29216/3920582546.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_vars] = encoder.fit_transform(X[cat_vars])


In [46]:
X.dtypes

age         float64
year           int8
collgrad       int8
race        float64
msp         float64
not_smsa    float64
c_city      float64
south       float64
dtype: object

In [87]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, mean_squared_error)
# split into sample A and sample B
X_A, X_B, D_A, D_B, Y_A, Y_B = train_test_split(X, D ,Y, test_size = 0.5, random_state = 123)


# Within each sample, make a validation set for xgboost early stopping
X_A_train, X_A_val, D_A_train, D_A_val, Y_A_train, Y_A_val = train_test_split(X_A, D_A, Y_A, test_size = 0.2, random_state = 123)
X_B_train, X_B_val, D_B_train, D_B_val, Y_B_train, Y_B_val = train_test_split(X_B, D_B, Y_B, test_size = 0.2, random_state = 123)

eval_set_A_Y = [(X_A_val, Y_A_val)]
eval_set_B_Y = [(X_B_val, Y_B_val)]

eval_set_A_D = [(X_A_val, D_A_val)]
eval_set_B_D = [(X_B_val, D_B_val)]

In [143]:
# Step 1. In both samples, train an xgboost regressor model to predict log wages (outcome Y)
# use early stopping.
from xgboost import XGBRegressor
xgbr_a = XGBRegressor()
xgbr_b = XGBRegressor()

xgbr_a.fit(X_A_train, Y_A_train, early_stopping_rounds = 20, eval_set = eval_set_A_Y, verbose = True)
xgbr_b.fit(X_B_train, Y_B_train, early_stopping_rounds = 10, eval_set = eval_set_B_Y, verbose = True)


print('RMSE on Test Set')
print('Sample A: %f' % np.sqrt(mean_squared_error(xgbr_a.predict(X_A_val), Y_A_val)))
print('Sample B: %f' % np.sqrt(mean_squared_error(xgbr_b.predict(X_B_val), Y_B_val)))

[0]	validation_0-rmse:0.95466
[1]	validation_0-rmse:0.72517
[2]	validation_0-rmse:0.58049
[3]	validation_0-rmse:0.49391
[4]	validation_0-rmse:0.44447
[5]	validation_0-rmse:0.41859
[6]	validation_0-rmse:0.40576
[7]	validation_0-rmse:0.39887
[8]	validation_0-rmse:0.39582
[9]	validation_0-rmse:0.39411
[10]	validation_0-rmse:0.39315
[11]	validation_0-rmse:0.39278
[12]	validation_0-rmse:0.39308
[13]	validation_0-rmse:0.39303
[14]	validation_0-rmse:0.39345
[15]	validation_0-rmse:0.39408
[16]	validation_0-rmse:0.39445
[17]	validation_0-rmse:0.39485
[18]	validation_0-rmse:0.39467
[19]	validation_0-rmse:0.39492
[20]	validation_0-rmse:0.39502
[21]	validation_0-rmse:0.39619
[22]	validation_0-rmse:0.39626
[23]	validation_0-rmse:0.39658
[24]	validation_0-rmse:0.39675
[25]	validation_0-rmse:0.39705
[26]	validation_0-rmse:0.39766
[27]	validation_0-rmse:0.39776
[28]	validation_0-rmse:0.39779


C:\Users\joppe\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[29]	validation_0-rmse:0.39797
[30]	validation_0-rmse:0.39796
[0]	validation_0-rmse:0.95057
[1]	validation_0-rmse:0.72435
[2]	validation_0-rmse:0.58136
[3]	validation_0-rmse:0.49627
[4]	validation_0-rmse:0.44913
[5]	validation_0-rmse:0.42299
[6]	validation_0-rmse:0.40939
[7]	validation_0-rmse:0.40292
[8]	validation_0-rmse:0.39882
[9]	validation_0-rmse:0.39724
[10]	validation_0-rmse:0.39636
[11]	validation_0-rmse:0.39576
[12]	validation_0-rmse:0.39531
[13]	validation_0-rmse:0.39593
[14]	validation_0-rmse:0.39666
[15]	validation_0-rmse:0.39639
[16]	validation_0-rmse:0.39709
[17]	validation_0-rmse:0.39726
[18]	validation_0-rmse:0.39783
[19]	validation_0-rmse:0.39782
[20]	validation_0-rmse:0.39857
[21]	validation_0-rmse:0.39917
[22]	validation_0-rmse:0.39965
RMSE on Test Set
Sample A: 0.392779
Sample B: 0.395315


In [135]:
# Step 2. In both samples, train an xgboost classifier model to predict union status (treatment D)
# use early stopping.
from xgboost import XGBClassifier

xgbc_a = XGBClassifier(n_estimators = 100, alpha = 0.01, max_depth =20, learning_rate = 0.3)
xgbc_b = XGBClassifier(n_estimators = 100, alpha = 0.01, max_depth = 20, learning_rate = 0.3)

xgbc_a.fit(X_A_train, D_A_train, early_stopping_rounds = 10, eval_set = eval_set_A_D, verbose = True)
xgbc_b.fit(X_B_train, D_B_train, early_stopping_rounds = 10, eval_set = eval_set_B_D, verbose = True)



print('F1 Score on Test Set')
print('Sample A: %f' % f1_score(xgbc_a.predict(X_A_val), D_A_val))
print('Sample B: %f' % f1_score(xgbc_b.predict(X_B_val), D_B_val))

# Unfortunately, the classification does not work very well, as seen from the F1 scores. With tweaking of the hyperparmaters
# not cross validated, but based on the validation set, we can get somewhat better performance, but still pretty poor...

[0]	validation_0-logloss:0.60743
[1]	validation_0-logloss:0.56783
[2]	validation_0-logloss:0.55015
[3]	validation_0-logloss:0.54025
[4]	validation_0-logloss:0.53795
[5]	validation_0-logloss:0.53797
[6]	validation_0-logloss:0.54117
[7]	validation_0-logloss:0.54377


C:\Users\joppe\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[8]	validation_0-logloss:0.54657
[9]	validation_0-logloss:0.55178
[10]	validation_0-logloss:0.55582
[11]	validation_0-logloss:0.56160
[12]	validation_0-logloss:0.56860
[13]	validation_0-logloss:0.57379
[0]	validation_0-logloss:0.61791
[1]	validation_0-logloss:0.58093
[2]	validation_0-logloss:0.56488
[3]	validation_0-logloss:0.55580
[4]	validation_0-logloss:0.55234
[5]	validation_0-logloss:0.55487
[6]	validation_0-logloss:0.55938
[7]	validation_0-logloss:0.56460
[8]	validation_0-logloss:0.57145
[9]	validation_0-logloss:0.57735
[10]	validation_0-logloss:0.58198
[11]	validation_0-logloss:0.58831
[12]	validation_0-logloss:0.59386
[13]	validation_0-logloss:0.59960
[14]	validation_0-logloss:0.60521
F1 Score on Test Set
Sample A: 0.215962
Sample B: 0.213636


In [69]:
# Step 3. Cross-fitting: Form predictions in other sample.

# predict wages in sample A using model trained in sample B:
Y_hat_A = xgbr_b.predict(X_A)

# vice versa:
Y_hat_B = xgbr_a.predict(X_B)
    
# predict union status in sample A using model trained in sample B:
D_hat_A = xgbc_b.predict(X_A)
    
# vice versa:
D_hat_B = xgbc_a.predict(X_B)
    

Recall that residuals for a variable $Z_i$ are computed as follows:

<center>$\tilde{Z}_i = Z_i - \hat{Z}_i$</center>

where $\hat{Z}_i$ is the predicted value of $Z_i$

In [77]:
# Compute residuals for wages and union status.

# residualized wages in samples A and B:
Y_tilde_A =  Y_A - Y_hat_A
Y_tilde_B =  Y_B - Y_hat_B

# residualized union status in samples A and B:
D_tilde_A =  D_A - D_hat_A#TODO
D_tilde_B =  D_B - D_hat_B#TODO

In [86]:
# Step 4. Run OLS regressions of the residualized variables and produce Double ML estimate
## use robust standard errors 
import statsmodels.regression.linear_model as smf

res_A = smf.OLS(Y_tilde_A, D_tilde_A).fit(cov_type = "hc1")
beta_A = res_A.params
se_A = res_A.bse

res_B = smf.OLS(Y_tilde_B, D_tilde_B).fit(cov_type = "hc1")
beta_B = res_B.params
se_B = res_B.bse

##Take the average of the two coefficient and standard errors and show the result

beta_average = (beta_A + beta_B)/2
print(beta_average)

se_average = (se_A + se_B)/2
print(se_average)

union    0.14345
dtype: float64
union    0.009681
dtype: float64


In [84]:
beta_B

union    0.130426
dtype: float64

**Compare the estimates form this and the previous regression. How do these change?**

When we compare the average value of the coefficient in the double ML approach to the normal OLS estimate, we see that it is smaller (0.14 vs 0.25). In general, we know that the estimate for beta in the normal OLS specification is biased. Now, we see that if we correct for this bias, using double ML, that the estimate for beta is indeed different. 

**Interpret the coefficient from the double ml approach. Can it be interpreted as causal?**

This coefficient can be interpreted as causal as long as our adjusting for covariates covers all relevant confounders. IF that is the case, then we can interpret beta as the causal effect. This is similar to what we discussed for regular adjusting for confounders. However, now we do it in a non-linear way, instead of linearly adjustment.

## Exercise 2: Synthetic Control Method

In this exercise we will investigate the economic effects of conflict by focusing on the Basque Country. In particular, we will focus on terrorist conflicts in late 1960's in this area and use as comparison group all other regions of Spain that did not experience terrorism. The (raw) reference regression is the following:

<center>$gdpcap_r = \beta_0 + \beta_1 terrorism_r + \varepsilon_r$</center>

where $gdpcap_r$ is GDP per capita in region $r$ and $terrorism_r$ is a dummy equal to 1 if terrorist conflict happened in that region.

In [ ]:
##note that to work correctly you will also have to downgrade scipy to version 1.4.1
!pip install scipy==1.4.1
!pip install SyntheticControlMethods

#restart the kernel after downrgading scipy

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_csv('data/HW06_data_scm.csv')
del df['Unnamed: 0']
del df['regionno']
df = df.loc[df["regionname"] != "Spain (Espana)"] 

df['terrorism'] = df['regionname']=='Basque Country (Pais Vasco)'
df['terrorism'] = df['terrorism'].astype('int')
df.head()

,regionname,year,gdpcap,sec.agriculture,sec.energy,sec.industry,sec.construction,sec.services.venta,sec.services.nonventa,school.illit,school.prim,school.med,school.high,school.post.high,popdens,invest,terrorism
43,Andalucia,1955,1.688732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
44,Andalucia,1956,1.758498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
45,Andalucia,1957,1.827621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
46,Andalucia,1958,1.852756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
47,Andalucia,1959,1.878035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [2]:
import seaborn as sns

##plot gdp per capita over time separately for basque country and other regions of spain
## (i.e., separately for terrorism == 1 or 0)

**Does the parallel trend assumption hold? If not, why do you think this is the case?**

In [ ]:
## run a regression of terrorism on gdp per capita

**What could be potential threats to a causal interpretation of this result?**

In [3]:
from SyntheticControlMethods import Synth, DiffSynth
del df['terrorism']

##fit the synthetic control 


In [ ]:
##visualize the plot 


**What can you conclude about the effect of conflict on GDP?**

In [8]:
## Show which spanish regions were used to create the synthetic control

## Exercise 3: Heterogenous Treatment Effects

In this exercise we will investigate the effect of case management on mental health outcomes. These data come from a randomized control trial where patients were assigned to *intensive* or *standard* case management. In this context, the treatment is being assigned to the **intensive** case management while patients assigned to the **standard** case management belong to the control group..

In this exercise we will focus on heterogenous treatment effects, rather than on isolating the casual effect of the treatment. In particular, we will investigate characteristics of individuals who are most and least responsive to the treatment, i.e., to being assigned to the intensive case management. 

In [1]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta')
df = df.dropna()
pd.read_stata('http://www.homepages.ucl.ac.uk/~rmjwiww/stata/missing/uk500.dta', iterator=True).variable_labels()

{'trialid': 'Trial ID',
 'centreid': 'Trial centre',
 'status': 'Patient status at baseline',
 'age': 'Age in years at baseline',
 'sex': 'Sex',
 'afcarib': 'Ethnic group',
 'ocfabth': "Father's social class at birth",
 'chron1l': 'Months since onset of psychosis, logged',
 'hos94': 'Days in hospital for psychiatric reasons: 2 years before baseline',
 'cprs94': 'Psychopathology at baseline (CPRS)',
 'das94': 'Disability at baseline (DAS)',
 'sat94': '(Dis)satisfaction with services at baseline',
 'rand': 'Randomised group',
 'hos96': 'Days in hospital for psychiatric reasons: 2 years after baseline',
 'cprs96': 'Psychopathology at 2 years (CPRS)',
 'sat96': '(Dis)satisfaction with services at 2 years'}

In [2]:
df.head()

,trialid,centreid,status,age,sex,afcarib,ocfabth,chron1l,hos94,cprs94,das94,sat94,rand,hos96,cprs96,sat96
1,107.0,St George's,Out-patient,27.0,male,Other,A,3.178054,80.0,4.0,0.285714,18.0,Intensive case management,27.0,3.0,22.000
2,222005.0,St Mary's,In hospital,41.0,male,Other,D,4.521789,240.0,6.0,0.750000,15.0,Intensive case management,15.0,13.0,9.000
3,222018.0,St Mary's,In hospital,25.0,male,Other,C2,4.094345,48.0,12.0,0.125000,18.0,Intensive case management,263.0,6.0,21.375
5,312015.0,King's,Out-patient,31.0,female,Other,A,4.787492,60.0,28.0,2.375000,20.0,Intensive case management,45.0,19.0,17.000
6,221023.0,St Mary's,In hospital,35.0,male,Afro-Caribbean,C2,4.430817,60.0,25.0,1.571428,24.0,Intensive case management,58.0,27.0,19.125


The treatment variable is $rand$, the post-treatment outcomes are $hos96$, $cprs96$ and $sat96$.

In [3]:
treatvar = 'rand'
df[treatvar].value_counts()

Intensive case management    130
Standard case management     116
Name: rand, dtype: int64

In [4]:
outcomes = ['sat96', 'hos96', 'cprs96']
df[outcomes].describe()

,sat96,hos96,cprs96
count,246.000000,246.000000,246.000000
mean,17.271341,65.500000,17.790587
std,4.723009,104.046722,14.090911
min,9.000000,0.000000,0.000000
25%,14.000000,0.000000,7.000000
50%,17.000000,15.000000,15.000000
75%,20.187500,93.500000,26.000000
max,32.000000,692.000000,71.000000


In addition to these variables we need a set of covariates that we want to use to identify individuals who are most and least responsive to treatment. We also encode categorical covariates and prepare them for the ML model

In [5]:
#Encoding Categorical covariates and preparing the data for tensorflow
covariates = ['status', 'sex', 'sat94', 'ocfabth', 'hos94', 'das94', 'cprs94', 'age', 'afcarib']
covariates_cat = ['status', 'sex', 'ocfabth', 'afcarib']

from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df[covariates_cat] = encoder.fit_transform(df[covariates_cat])
df[covariates] = df[covariates].astype('float32')
df[covariates].head()

,status,sex,sat94,ocfabth,hos94,das94,cprs94,age,afcarib
1,1.0,1.0,18.0,0.0,80.0,0.285714,4.0,27.0,1.0
2,0.0,1.0,15.0,4.0,240.0,0.750000,6.0,41.0,1.0
3,0.0,1.0,18.0,3.0,48.0,0.125000,12.0,25.0,1.0
5,1.0,0.0,20.0,0.0,60.0,2.375000,28.0,31.0,1.0
6,0.0,1.0,24.0,3.0,60.0,1.571428,25.0,35.0,0.0


In [ ]:
# Subset the dataset by treatment and control
# Within each sample, create a training, a test and a validation set

df_treat = #TODO
df_control = #TODO

In [ ]:
##choose one of the three outcomes to analyze
## using xgboost regressor train two model to predict the outcome from the covariates
## the first model should be trained on the treated sample, while the second on the control
## use early stopping

from xgboost import XGBRegressor
xgbr_treat = XGBRegressor()
xgbr_control = XGBRegressor()

In [ ]:
##get predicted outcomes using the combined test sets for both models 

test_set = Xtreat_test.append(Xcontrol_test) 
y_hat_treat = ...
y_hat_control = ...

In [ ]:
##find the three individuals in the test set that are most and least responsive to the treatment
##namely the three individuals for who the treatment effect is larger and those for who it is smaller

test_set['treat_effect'] = ...

In [ ]:
##visualize and comment on the covariates of these individuals 